In [1]:
pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=de9b800215e99e50937a9b0e512940a03ed4b7dc2cd7ea3138eb1bfec65c1dc7
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [2]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 4.2 MB/s 
     |████████████████████████████████| 212 kB 58.1 MB/s 
     |████████████████████████████████| 1.1 MB 62.7 MB/s 
     |████████████████████████████████| 86 kB 8.2 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 140 kB 67.5 MB/s 
     |████████████████████████████████| 596 kB 53.9 MB/s 
     |████████████████████████████████| 127 kB 70.0 MB/s 
     |████████████████████████████████| 271 kB 48.0 MB/s 
     |████████████████████████████████| 94 kB 1.0 MB/s 
     |████████████████████████████████| 144 kB 55.1 MB/s 
     |████████████████████████████████| 112 kB 47.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing ins

In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 28.7 MB/s 


In [4]:
import pickle
import numpy as np
import transformers
from transformers import Trainer
from datasets import load_metric
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification


In [5]:
raw_dataset = load_dataset('surdan/nerel_short')

Downloading:   0%|          | 0.00/924 [00:00<?, ?B/s]

Using custom data configuration surdan--for_test_v3-eb84b899e14cc88e


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/surdan___parquet/surdan--for_test_v3-eb84b899e14cc88e/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
print(raw_dataset)

DatasetDict({
    test: Dataset({
        features: ['sequences', 'ids'],
        num_rows: 512
    })
    dev: Dataset({
        features: ['sequences', 'ids'],
        num_rows: 536
    })
    train: Dataset({
        features: ['sequences', 'ids'],
        num_rows: 2508
    })
})


In [7]:
item = raw_dataset['train'][0] # returns a dict object
print(item['sequences'])
print(item['ids'])

['Виталий', 'Кличко', 'хочет', 'стать', 'мэром', 'Киева', 'Бывший', 'чемпион', 'мира', 'по', 'боксу', 'в', 'тяжёлом', 'весе', 'Виталий', 'Кличко', 'в', 'марте', 'будет', 'баллотироваться', 'на', 'пост', 'мэра', 'столицы', 'Украины', 'Киева']
[44, 45, 0, 0, 48, 5, 0, 3, 4, 0, 0, 0, 0, 0, 44, 45, 11, 12, 0, 0, 0, 0, 48, 0, 7, 5]


In [8]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
inputs = tokenizer(raw_dataset["train"][0]["sequences"], is_split_into_words=True)
print(inputs.tokens())

['[CLS]', 'Виталий', 'Кличко', 'хочет', 'стать', 'мэром', 'Киева', 'Бывший', 'чемпион', 'мира', 'по', 'боксу', 'в', 'тяжёлом', 'весе', 'Виталий', 'Кличко', 'в', 'марте', 'будет', 'баллотироваться', 'на', 'пост', 'мэра', 'столицы', 'Украины', 'Киева', '[SEP]']


In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["sequences"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ids"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
tokenized_dataset= raw_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_dataset["train"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [12]:
print(tokenized_dataset)

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 512
    })
    dev: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 536
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2508
    })
})


In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
with open('/content/drive/MyDrive/Colab Notebooks/MLcourse/NLP/id_to_label_map.pickle', 'rb') as f:
    map_id_to_label = pickle.load(f)

In [16]:
id2label = {str(k): v for k, v in map_id_to_label.items()}
label2id = {v: k for k, v in id2label.items()}
label_names = list(id2label.values())

In [27]:
model = AutoModelForTokenClassification.from_pretrained("DeepPavlov/rubert-base-cased-sentence", id2label=id2label, label2id=label2id)

loading configuration file https://huggingface.co/DeepPavlov/rubert-base-cased-sentence/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c6da12aa84b0056f7fcf2ce40343ab08fd71045914c431fd8bf57709efc5abae.e8f15c5aad2f4653e46ceeba0bb32c02a629d106a902c964bce60523d290ac8f
Model config BertConfig {
  "_name_or_path": "DeepPavlov/rubert-base-cased-sentence",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "I-AGE",
    "10": "I-CRIME",
    "11": "B-DATE",
    "12": "I-DATE",
    "13": "B-DISEASE",
    "14": "I-DISEASE",
    "15": "B-DISTRICT",
    "16": "I-DISTRICT",
    "17": "B-EVENT",
    "18": "I-EVENT",
    "19": "B-FACILITY",
    "2": "B-AGE",
    "20": "I-FACILITY",
    "21": "B-FAMILY",
    "22": "I-FAMILY",
    "23": "B-IDEOLOGY",
    "24": "I-IDEOLOGY",
 

In [18]:
## for compute_metrics function
from datasets import load_metric
metric = load_metric("seqeval")

In [19]:
def compute_metrics(eval_preds):
    """
    Function for evaluate model
    
    :param eval_preds: model output
    :type eval_preds: 
    """
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_names[p] for (p, l) in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)
                        ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [28]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    per_device_train_batch_size = 4 ## depending on the total volume of memory of your GPU
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["dev"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model('./saved_model') #For reuse

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2508
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1881


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.975000,0.606300,0.659367,0.721268,0.688930,0.854276
2,0.403300,0.532337,0.714620,0.739173,0.726689,0.873557
3,0.268400,0.528344,0.733049,0.748125,0.740510,0.878271


***** Running Evaluation *****
  Num examples = 536
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 536
  Batch size = 8
***** Running Evaluation *****
  Num examples = 536
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./saved_model
Configuration saved in ./saved_model/config.json
Model weights saved in ./saved_model/pytorch_model.bin
tokenizer config file saved in ./saved_model/tokenizer_config.json
Special tokens file saved in ./saved_model/special_tokens_map.json


In [29]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 536
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 3.0,
 'eval_accuracy': 0.8782713810818978,
 'eval_f1': 0.7405101185486767,
 'eval_loss': 0.5283443331718445,
 'eval_precision': 0.7330488383119962,
 'eval_recall': 0.7481248487781272,
 'eval_runtime': 3.5971,
 'eval_samples_per_second': 149.009,
 'eval_steps_per_second': 18.626}

In [30]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/saved_model"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)

loading configuration file /content/saved_model/config.json
Model config BertConfig {
  "_name_or_path": "/content/saved_model",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "I-AGE",
    "2": "B-AGE",
    "3": "B-AWARD",
    "4": "I-AWARD",
    "5": "B-CITY",
    "6": "I-CITY",
    "7": "B-COUNTRY",
    "8": "I-COUNTRY",
    "9": "B-CRIME",
    "10": "I-CRIME",
    "11": "B-DATE",
    "12": "I-DATE",
    "13": "B-DISEASE",
    "14": "I-DISEASE",
    "15": "B-DISTRICT",
    "16": "I-DISTRICT",
    "17": "B-EVENT",
    "18": "I-EVENT",
    "19": "B-FACILITY",
    "20": "I-FACILITY",
    "21": "B-FAMILY",
    "22": "I-FAMILY",
    "23": "B-IDEOLOGY",
    "24": "I-IDEOLOGY",
    "25": "B-LANGUAGE",
    "26": "I-LAW",
    "27": "B-LAW",
    "28": "B-LOCATION",
    "29"

In [31]:
token_classifier("Отвечая на вопрос, кто принял решение о закрытии воздушного пространства Черногории для борта Лаврова, Кривокапич отметил, что формально решение принято правительством и министерством иностранных дел страны.")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'end': 72,
  'entity_group': 'EVENT',
  'score': 0.65745026,
  'start': 40,
  'word': 'закрытии воздушного пространства'},
 {'end': 83,
  'entity_group': 'COUNTRY',
  'score': 0.97847146,
  'start': 73,
  'word': 'Черногории'},
 {'end': 101,
  'entity_group': 'PERSON',
  'score': 0.9880172,
  'start': 94,
  'word': 'Лаврова'},
 {'end': 113,
  'entity_group': 'PERSON',
  'score': 0.965189,
  'start': 103,
  'word': 'Кривокапич'},
 {'end': 167,
  'entity_group': 'ORGANIZATION',
  'score': 0.977671,
  'start': 153,
  'word': 'правительством'},
 {'end': 199,
  'entity_group': 'ORGANIZATION',
  'score': 0.97538394,
  'start': 170,
  'word': 'министерством иностранных дел'}]